In [1]:
import os


In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\End-to_End_MLflow'

In [4]:
import tensorflow as tf



In [5]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [6]:
from dataclasses import dataclass
from pathlib import Path


In [7]:
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from cnn_Classifier.constants import *
from cnn_Classifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(self, config_filepath:str=CONFIG_FILE_PATH, params_filepath:str=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/Data",
            all_params=self.params,
            mlflow_uri="https://dagshub.com/Anhtt9x/End-to_End_MLflow.mlflow",
            params_batch_size=self.params.BATCH_SIZE,
            params_image_size=self.params.IMAGE_SIZE
        )

        return eval_config

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [14]:
class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        data_generator_kwargs = dict(
            rescale= 1/255,
            validation_split=0.3
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )

        self.valid_generator = valid_data_generator.flow_from_directory(
            directory= self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss":self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_registry_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0],
                 "accuracy": self.score[1]}
            )
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(model=self.model, artifact_path="model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(model=self.model , artifact_path="model")

In [12]:
import dagshub
dagshub.init(repo_owner='Anhtt9x', repo_name='End-to_End_MLflow', mlflow=True)


Initialized MLflow to track repo "Anhtt9x/End-to_End_MLflow"

[2024-05-27 08:16:26,393: INFO : helpers : Initialized MLflow to track repo "Anhtt9x/End-to_End_MLflow"]


Repository Anhtt9x/End-to_End_MLflow initialized!

[2024-05-27 08:16:26,398: INFO : helpers : Repository Anhtt9x/End-to_End_MLflow initialized!]


In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation()
    evaluation = Evaluation(config=eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise(e)


[2024-05-27 08:16:26,429: INFO : common : yaml file: D:\End-to_End_MLflow\config\config.yaml loaded successfully]
[2024-05-27 08:16:26,441: INFO : common : yaml file: D:\End-to_End_MLflow\params.yaml loaded successfully]
[2024-05-27 08:16:26,444: INFO : common : Create directory: artifacts]


Found 298 images belonging to 3 classes.


d:\End-to_End_MLflow\mlflow\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.0486 - loss: 3.9336


2024/05/27 08:17:17 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
d:\End-to_End_MLflow\mlflow\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/05/27 08:18:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
